In [1]:
import librosa
import pyAudioAnalysis as pyaudio
import numpy as np
import pandas as pd
import os

In [2]:
from pyAudioAnalysis import audioBasicIO
from pyAudioAnalysis import MidTermFeatures

In [5]:
features = []

In [6]:
for song in os.scandir("."):
    if song.path.endswith(".mp3") and song.is_file():
        # skip ./ in the path to the mp3 file
        file_name = song.path[2:]
        
        # get the tempo of the song
        waveform, samp_rate = librosa.load(file_name)
        tempo, beat_frames = librosa.beat.beat_track(waveform, samp_rate)
        
        # get the chroma number of the song
        beat_times = librosa.frames_to_time(beat_frames, samp_rate)
        y_harmonic, y_percussive = librosa.effects.hpss(waveform)
        chromagram = librosa.feature.chroma_cqt(y_harmonic, samp_rate)
        beat_chroma = librosa.util.sync(chromagram, beat_frames, aggregate=np.median)
        # make beat chroma into a DataFrame and calculate the diff
        chroma_df = pd.DataFrame(beat_chroma)
        diff_values = chroma_df.diff()
        diff_mean = diff_values.mean(axis = 0, skipna = True)
        chroma_num = sum(diff_mean) / len(diff_mean)
        
        # add song, tempo and chroma number to the features list
        song_name = file_name[:len(file_name) - 4]
        print ([song_name, tempo, chroma_num])
        features.append([song_name, tempo, chroma_num])

/opt/anaconda3/envs/music/lib/python3.8/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


['red - taylor swift', 123.046875, 0.033330368142967165]


/opt/anaconda3/envs/music/lib/python3.8/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


["it's you - henry", 151.99908088235293, -0.0013526432818837847]


/opt/anaconda3/envs/music/lib/python3.8/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


...

['wake me up - avicii', 123.046875, 0.026902930808876502]


/opt/anaconda3/envs/music/lib/python3.8/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


['super clap - super junior', 129.19921875, -0.005076935539705365]


/opt/anaconda3/envs/music/lib/python3.8/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


['i - taeyeon, verbal jint', 89.10290948275862, 0.015557589302528261]


In [7]:
features_df = pd.DataFrame(features, columns = ['song', 'tempo', 'chroma_number'])
features_df = features_df.sort_values(by = ['song'])

In [9]:
mid_term_window = 1
mid_term_step = 1
short_term_window = 0.05
short_term_step = 0.05
pyaudio_feat, files, feat_names = MidTermFeatures.directory_feature_extraction(".", 
                                                                               mid_term_window, 
                                                                               mid_term_step, 
                                                                               short_term_window, 
                                                                               short_term_step)

Analyzing file 1 of 71: ./a oh - super junior.mp3
Analyzing file 2 of 71: ./alliance force, assemble - takanashi yasuharu.mp3
Analyzing file 3 of 71: ./angela - super junior.mp3
Analyzing file 4 of 71: ./bad things - camila cabello, machine gun kelly.mp3
Analyzing file 5 of 71: ./because i miss you - kyuhyun.mp3
Analyzing file 6 of 71: ./beggin on your knees - victoria justice.mp3
Analyzing file 7 of 71: ./between - yesung.mp3
Analyzing file 8 of 71: ./bie qing yi shuo ai - cheng xin.mp3
Analyzing file 9 of 71: ./cassandra - two steps from hell.mp3
Analyzing file 10 of 71: ./celestial spirit leo - takanashi yasuharu.mp3
Analyzing file 11 of 71: ./city - yamagami takeshi.mp3
Analyzing file 12 of 71: ./cosmic - ryeowook, bada.mp3
Analyzing file 13 of 71: ./dive to world - cherryblossom.mp3
Analyzing file 14 of 71: ./do you hear what i hear - carrie underwood.mp3
Analyzing file 15 of 71: ./don't stop - 5 seconds of summer.mp3
Analyzing file 16 of 71: ./dragon rider - two steps from hell.m

In [33]:
# neural network features
features_df.reset_index(drop = True, inplace = True)
feat_names.append("unknown_column_1")
feat_names.append("unknown_column_2")
nn_features_df = pd.DataFrame(pyaudio_feat, columns = feat_names)
nn_features_df['song'] = features_df['song']
nn_features_df.to_csv("../data/nn_features.csv", header = True, index = False)

In [31]:
zcr_ind = feat_names.index('zcr_mean')
ee_ind = feat_names.index('energy_entropy_mean')
spc_ind = feat_names.index('spectral_centroid_mean')

features_df['zero_crossing_rate'] = nn_features_df.iloc[:, zcr_ind]
features_df['energy_entropy'] = nn_features_df.iloc[:, ee_ind]
features_df['spectral_centroid'] = nn_features_df.iloc[:, spc_ind]

In [34]:
features_df

,song,tempo,chroma_number,zero_crossing_rate,energy_entropy,spectral_centroid
0,a oh - super junior,129.199219,-0.005782,0.068616,3.249814,0.153889
1,"alliance force, assemble - takanashi yasuharu",95.703125,0.007708,0.069429,3.120086,0.166438
2,angela - super junior,107.666016,-0.001115,0.072561,3.172593,0.170664
3,"bad things - camila cabello, machine gun kelly",135.999178,0.013476,0.060101,3.212073,0.143173
4,because i miss you - kyuhyun,73.828125,0.023379,0.057730,3.191261,0.136981
...,...,...,...,...,...,...
66,why don't we go there - one direction,161.499023,-0.013630,0.054871,3.204648,0.124632
67,wo shi zhen de ai guo ni - liu zeng tong,143.554688,0.000571,0.079562,3.226773,0.185206
68,"wolves - selena gomez, marshmello",123.046875,0.025086,0.088343,3.207983,0.179641
69,you he bu ke - xu song,99.384014,0.015339,0.076929,3.196292,0.164416


In [35]:
features_df.to_csv("../data/features.csv", header = True, index = False)